In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [9]:
len(os.listdir(os.path.join('./datasets/AIROGS_partial/',os.listdir('./datasets/AIROGS_partial/')[0])))

164

In [10]:
# /models/teacher_experiments/
# m = load_model('./models/teacher_experiments/EffNet_B7_imagenet_norescale_sameastfmodel_rev.h5')
# new_model = Model(inputs = m.inputs, outputs = m.layers[-2].output, name = 'Effnet_b7_exp17_logits')
# new_model.save('./models/teacher_experiments/EffNet_B7_exp17_NoActivation.h5')

In [11]:
# m.summary()

In [12]:
# new_model.summary()

In [18]:
# Importing necessary libraries
from PIL import Image, ImageFilter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.python import keras
from tensorflow import keras
# import tensorflow_hub as hub
# from tensorflow.keras.applications import EfficientNetB4, MobileNetV3Small
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, save_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau, TensorBoard, Callback, ProgbarLogger
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import *
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
import os

tf.keras.backend.clear_session()

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')
# mixed_precision.set_global_policy('float32')
import warnings
warnings.filterwarnings('ignore')

In [19]:
from scipy import stats
dfscratch = pd.read_csv('./performance_stat_significance_comparision/Performance_Student.csv')#, header=True)
dfdist = pd.read_csv('./performance_stat_significance_comparision/Performance_Distillation.csv')#, header=True)

In [20]:
# dfscratch.head()
metrics = ['AUC','Qwk','Accuracy']
datasets = ['IDRID','Kaggle','Messidor2']
# dfscratch_reshaped = pd.Dataframe(columns=['Dataset','Model Name','Scratch_IDRID','Scratch_Kaggle','Scratch_Messidor2','Metric'])
# dfdistreshaped = pd.Dataframe(columns=['Dataset','Model Name','Dist_IDRID','Dist_Kaggle','Dist_Messidor2','Metric'])
dfscratch_reshaped = pd.DataFrame()
dfdist_reshaped = pd.DataFrame()
for metric in metrics:
    df_reshaped1 = dfscratch.pivot(index='Model Name', columns='Dataset', values=metric).reset_index()
    df_reshaped1['Metric'] = metric
    for column in df_reshaped1.columns:
        if column in datasets:
            df_reshaped1.rename(columns={column:'Scratch_'+column}, inplace=True)
    dfscratch_reshaped = dfscratch_reshaped.append(df_reshaped1)
    dfscratch_reshaped = dfscratch_reshaped.reset_index(drop=True)
    
    df_reshaped2 = dfdist.pivot(index='Model Name', columns='Dataset', values=metric).reset_index()
    df_reshaped2['Metric'] = metric
    for column in df_reshaped2.columns:
        if column in datasets:
            df_reshaped2.rename(columns={column:'Dist_'+column}, inplace=True)
    dfdist_reshaped = dfdist_reshaped.append(df_reshaped2)
    dfdist_reshaped = dfdist_reshaped.reset_index(drop=True)

dfscratch_reshaped = dfscratch_reshaped.rename_axis(None, axis=1)
dfdist_reshaped = dfdist_reshaped.rename_axis(None, axis=1)
dffinal = pd.merge(dfscratch_reshaped,dfdist_reshaped, on=['Model Name','Metric'])

scratch_columns = ['Scratch_IDRID', 'Scratch_Kaggle', 'Scratch_Messidor2']
dist_columns = ['Dist_IDRID', 'Dist_Kaggle', 'Dist_Messidor2']

# Perform paired t-test and add results to DataFrame
dffinal['T-statistic'], dffinal['P-value'] = zip(*dffinal.apply(
    lambda row: stats.ttest_rel(row[dist_columns],row[scratch_columns],alternative='greater'), axis=1))
# dffinal['Improvement (99% Confidence)'] = dffinal.apply(lambda row: 'significant' if row['T-statistic'] > 0 and row['P-value'] <= 0.01 else 'not significant', axis=1)
dffinal['Improvement (95% Confidence)'] = dffinal.apply(lambda row: 'significant' if row['T-statistic'] > 0 and row['P-value'] <= 0.05 else 'not significant', axis=1)
dffinal['Improvement (90% Confidence)'] = dffinal.apply(lambda row: 'significant' if row['T-statistic'] > 0 and row['P-value'] <= 0.1 else 'not significant', axis=1)
# dffinal['Improvement (80% Confidence)'] = dffinal.apply(lambda row: 'significant' if row['T-statistic'] > 0 and row['P-value'] <= 0.2 else 'not significant', axis=1)
# dffinal.head(15)
dffinal.to_csv('Stat_Sig_Improved_Distillation_Performance.csv',header=True, index=False)